In [18]:
import numpy as np
import tensorflow as tf
import pickle
import os
from tensorflow.contrib import layers
from tensorflow.contrib import rnn

tf.set_random_seed(0)

In [2]:
position = 'right'
with open("C:\\ASM\\DevData\\accel2gyro\\data\\steven_free\\steven_free_"+position+".pkl", "rb") as file:
    data = pickle.load(file)
#print(data[0][0][:10, :3])

In [3]:
subject_count = len(data)    
for s in range(subject_count):
    sess_count = len(data[s])
    for sess in range(sess_count):
        d = data[s][sess]
        accel = d[:, :3]
        gyro = d[:, 3:]
        accel_diff = accel[1:, :] - accel[:-1, :]        
        data[s][sess] = np.concatenate((accel[1:, :], accel_diff, gyro[1:, :]), axis = 1)
        #print("Subject {}, Sess {} : {}".format(s, sess, data[s][sess].shape))

In [4]:
def get_data(indices):
    count = len(indices)
    x = np.zeros((count, 32, 6))
    y = np.zeros((count, 16, 3))
    for i in range(count):
        subject = indices[i][0]
        sess = indices[i][1]
        xa = indices[i][2]
        xb = indices[i][3]
        ya = indices[i][4]
        yb = indices[i][5]
        
        x[i, :, :] = data[subject][sess][xa:xb, :6]
        y[i, :, :] = data[subject][sess][ya:yb, 6:]
        
    res = {"x":x, "y":y}
    return res

def get_train_indices_test_data(test_subject):
    subject_count = len(data)
    train_ix = []
    test_data = []
    for s in range(subject_count):        
        sess_count = len(data[s])
        for sess in range(sess_count):            
            count = len(data[s][sess])            
            step_size = 8
            if s == test_subject:
                step_size = 16
            
            ix_accel_start = np.arange(0, count-32+1, step_size, dtype=np.int32).reshape(-1, 1)            
            ix_accel_end = (ix_accel_start + 32).reshape(-1, 1)
            ix_gyro_start = (ix_accel_start + 8).reshape(-1, 1)
            ix_gyro_end = (ix_accel_start + 24).reshape(-1, 1)
            
            ix_count = len(ix_accel_start)
            ix_s = np.zeros((ix_count, 1), dtype=np.int32) + s
            ix_sess = np.zeros((ix_count, 1), dtype=np.int32) + sess
            
            ix = np.concatenate((ix_s, ix_sess, ix_accel_start, ix_accel_end, ix_gyro_start, ix_gyro_end) , axis = 1)
            
            if s == test_subject:
                d = get_data(ix)
                test_data.append(d)
            else:
                if len(train_ix)==0:
                    train_ix = ix
                else:
                    train_ix = np.concatenate((train_ix, ix))
        
    np.random.shuffle(train_ix)        
    return train_ix, test_data
            

In [10]:
test_subject = 0
train_ix, test_data = get_train_indices_test_data(test_subject)    

In [16]:
SEQLEN_IN = 32
SEQLEN_OUT = 16
BATCH_SIZE = 128
INPUT_SIZE = 6
OUTPUT_SIZE = 3
INTERNAL_SIZE = 256
NLAYERS = 3
learning_rate = 0.001  # fixed learning rate
dropout_pkeep = 0.8    # some dropout
display_step = 1000

In [17]:
epoch_size = len(train_ix) // BATCH_SIZE
print("Epoch size: ", epoch_size)

Epoch size:  12231


In [ ]:
X = tf.placeholder(tf.float32, [None, SEQLEN_IN, INPUT_SIZE], name='X')    # [ BATCH_SIZE, SEQLEN, INPUT_SIZE ]
Y = tf.placeholder(tf.float32, [None, SEQLEN_OUT, OUTPUT_SIZE], name='Y')

fw_cells = [rnn.BasicLSTMCell(INTERNALSIZE) for _ in range(NLAYERS)]
bw_cells = [rnn.BasicLSTMCell(INTERNALSIZE) for _ in range(NLAYERS)]

multi_fw_cells = rnn.MultiRNNCell(fw_cells, state_is_tuple=False)
multi_bw_cells = rnn.MultiRNNCell(bw_cells, state_is_tuple=False)